# Исследование надежности заемщиков

Заказчик — кредитный отдел банка. Необходимо выяснить: влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. 

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.


**Цель исследования** - проверка гипотез:

1.	Количество детей заемщика влияет на возврат кредита в срок.
2.	Семейное положение заемщика влияет на возврат кредита в срок.
3.	Уровень дохода заемщика влияет на возврат кредита в срок.
4.	Как цель кредита влияет на его возврат в срок.


**Ход исследования**

Входные данные от банка — статистика о платёжеспособности клиентов из файла /datasets/data.csv. 
О качестве данных известно, что присутствуют пропущенные значения, аномалии и дубликаты строк. 
Поэтому перед проведением исследования потребуется обзор данных и их предобработка.


Исследование пройдёт в три этапа:
 1. Обзор данных.
 2. Предобработка данных.
 3. Проверка гипотез.

## Обзор данных

In [1]:
import pandas as pd

In [2]:
try:
    df = pd.read_csv('C:/Users/Anna/Desktop/Обучение/Яндекс практикум/Проект 2/data.csv') # чтение файла с данными и сохранение в df
    
except FileNotFoundError:
    df = pd.read_csv('/datasets/data.csv') # чтение файла с данными и сохранение в df


df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
df.info() # получение общей информации о данных в таблице df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Итак, в таблице двенадцать столбцов. 
Типы данных столбцов: int64, float64, object 

Согласно документации к данным:

* `children` — количество детей в семье
* `days_employed` — общий трудовой стаж в днях
* `dob_years` — возраст клиента в годах
* `education` — уровень образования клиента
* `education_id` — идентификатор уровня образования
* `family_status` — семейное положение
* `family_status_id` — идентификатор семейного положения
* `gender` — пол клиента
* `income_type` — тип занятости
* `debt` — имел ли задолженность по возврату кредитов
* `total_income` — ежемесячный доход
* `purpose` — цель получения кредита

Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.


**Выводы**

В каждой строке таблицы — данные о заемщике. Часть колонок описывает кредитоспособность заемщика: тип занятости, задолженность по возврату кредита, ежемесячный доход, цель получения кредита. Остальные данные рассказывают о самом заемщике: количество детей, трудовой стаж, возраст, образование, семейное положение, пол.

Предварительно можно утверждать, что, данных достаточно для проверки гипотез. Но встречаются пропуски в данных и некорректные значения.

Необходимо устранить проблемы в данных.

## Предобработка данных


### Проверка данных на аномалии и исправления.
В столбце `days_employed` данные представлены с ошибкой -  отрицательное количество дней трудового стажа.
В 5 строке столбца `days_employed` видим некорректное значение. Переводим трудовой стаж - 340266 / 365 = 932 года, что невозможно при возрасте клиента 53 года. Чтобы избавиться от отрицательных значений, возьмем модуль значений столбца `days_employed`.

In [4]:
df['days_employed'] = abs(df['days_employed']) #берем модуль столбца `days_employed` избавляемся от отрицательных значений

Для заполнения аномально больших данных в столбце `days_employed` рассчитаем медианное значение трудового стажа для каждого возраста

In [5]:
df.groupby('dob_years')['days_employed'].median() #расчитываем медианные значения для каждой возрастной группы

dob_years
0       1759.038033
19       724.492610
20       674.838979
21       618.733817
22       703.310078
23       690.204208
24       947.731043
25       919.199388
26      1083.658132
27      1166.212160
28      1141.705450
29      1315.453550
30      1420.586863
31      1308.901135
32      1446.622542
33      1426.415003
34      1615.910188
35      1613.494818
36      1799.520465
37      1816.713382
38      1817.194286
39      1891.388566
40      1728.936706
41      1864.657692
42      2253.981630
43      1869.019899
44      2084.330015
45      2254.431130
46      2100.473217
47      2203.078545
48      2429.674518
49      2560.317106
50      2626.986257
51      2846.080700
52      3395.639568
53      3650.007523
54      4026.541145
55      7043.471139
56      6850.179463
57    333953.491811
58    331245.976666
59    344372.337365
60    349414.537285
61    343984.931910
62    354981.718187
63    358482.295938
64    355781.562599
65    357103.967291
66    363827.438951
67    3572

Возрастной диапазон находится в пределах от 19 до 75, что соответствует логике, однако в данных присутствует значение возраста - 0 лет, что является ошибкой. Заменим аномальное значение 0 лет - возрастом ближе всего подходящим по медианным значениям трудового стажа. 

Аномально большие значения в данных характерны для возрастного диапазона 59-74 года.

In [6]:
''' Cгруппируем данные по медианному значению трудового стажа для каждой возрастной группы. 
Создадим функцию, в которой будем проверять корректность данных о возрасте заемщиков, возьмем начало трудовой деятельности
возраст 18 лет. Функция будет заменять значения некорректного возраста  - возрастом ближе всего подходящим по медиане 
трудового стажа'''

df_grouped = df.groupby('dob_years')['days_employed'].median()
 
def func(row):
    if row['dob_years'] < 18:
        return df_grouped.loc[row['dob_years']]
    return row['dob_years']
 
df['dob_years'] = df.apply(func, axis=1).astype('int') 
df.groupby('dob_years')['days_employed'].median().head(10)   

dob_years
19     724.492610
20     674.838979
21     618.733817
22     703.310078
23     690.204208
24     947.731043
25     919.199388
26    1083.658132
27    1166.212160
28    1141.705450
Name: days_employed, dtype: float64

Возможными причинами возникновения данных ошибок могут являтся: ошибки ввода информации, ошибки при копировании или переносе данных из других источников. Часть информации могла быть введена в годах, а другая в днях.

Для корректировки данных столбца `days_employed`, проверяем условие, что трудовой стаж не превышает возраст клиента с начала его трудовой деятельности.

In [7]:
# проверяем условие, что трудовой стаж не превышает возраст клиента с 18 лет, если условие не выполнено, то меняем значение
# столбца 'days_employed' на максимально возможный трудовой стаж исходя из возраста клиента

df.loc[(df['days_employed'] > (df['dob_years'] - 18)*365), 'days_employed'] = (df['dob_years'] - 18)*365

In [8]:
df.groupby('dob_years')['days_employed'].median() #медианное значение в каждой возрастной группе не превышает допустимые значения

dob_years
19        365.000000
20        674.838979
21        618.733817
22        703.310078
23        690.204208
24        947.731043
25        919.199388
26       1083.658132
27       1166.212160
28       1141.705450
29       1315.453550
30       1420.586863
31       1308.901135
32       1446.622542
33       1426.415003
34       1615.910188
35       1613.494818
36       1799.520465
37       1816.713382
38       1817.194286
39       1891.388566
40       1728.936706
41       1864.657692
42       2253.981630
43       1869.019899
44       2084.330015
45       2254.431130
46       2100.473217
47       2203.078545
48       2429.674518
49       2560.317106
50       2626.986257
51       2846.080700
52       3395.639568
53       3650.007523
54       4026.541145
55       7043.471139
56       6850.179463
57      14235.000000
58      14600.000000
59      14965.000000
60      15330.000000
61      15695.000000
62      16060.000000
63      16425.000000
64      16790.000000
65      17155.000000
66 

Проверим на аномалии столбцы со значениями необходимые нам для исследования

In [9]:
df['children'].unique() #проверяем значения столбца 'children'

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

In [10]:
df['children'].value_counts() #рассчитываем количество заемщиков по группам количества детей

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

В столбце `children` присутствует отрицательное значение количества детей, что не соответствует действительности.
Исправим отрицательные значения взяв модуль столбца `children`. В данных присутствует 76 строк со значение количества детей 20, что не соответствует действительности, т.к.  не известно какие точно данные должны быть в строке 0 или 2, замени недостающие данные медианным значением. Причиной возникновения данной ошибки предположительно является ошибка ввода информации.

In [11]:
df.loc[(df['children'] == 20), 'children'] = df['children'].median()

In [12]:
df['children'] = df['children'].abs().astype('int')
df['children'].unique() #проверяем произведенные изменения

array([1, 0, 3, 2, 4, 5])

In [13]:
df['gender'].unique() #проверяем значения столбца 'gender' 

array(['F', 'M', 'XNA'], dtype=object)

В столбце 'gender' обнаружено значение 'XNA', выведем все строки со значением 'XNA'.

In [14]:
df[df['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,2190.0,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


Строк со значением 'XNA' только 1, для проверки гипотез пол клиента нам не важен, заменим значение на 'M'. Скорее всего это тоже ошибка ввода информации или пропущенное значение.

In [15]:
df.loc[(df['gender'] == 'XNA'), 'gender'] = 'M'

In [16]:
df['gender'].unique() #проверяем внесенные изменения

array(['F', 'M'], dtype=object)

### Заполнение пропусков

Проверим наличие пропусков в данных


In [17]:
df.isna().sum() # подсчёт пропусков

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Обнаружено по 2174 пропущенных значения в столбцах `days_employed`-общий трудовой стаж в днях и `total_income`-ежемесячный доход.
Доля пропущенных значений в каждом из столбцов составляет 10%, что может существенно повлиять на выводы исследования.
Причинами пропуска данных могли явиться намеренное сокрытие информации. Т.к. число пропусков в обоих столбцах одиноково: следовательно данные пропуски не являются случайными. 

Процент пропусков данных составляет существенную часть и эти данные необходимы для исследования. Пропущенные данные в столбцах являются количественными переменными, соответственно для заполнения пропусков мы можем выбрать заполнение средними значениями или медианными.

In [18]:
#сравнеие среднего значения и медианы в столбце 'days_employed' и 'total_income' для выбора способа заполнения пропусков

df.groupby('debt').agg({'days_employed': ['mean', 'median'], 'total_income': ['mean', 'median']})

days_employed                total_income               
              mean       median           mean         median
debt                                                         
0      5044.225901  2258.679577  167804.131651  145161.906773
1      3518.702377  1495.893009  163100.897062  143302.999614

 При сравнение среднего арифметического и медианы их значения не совпадают, значит распределение несимметрично из-за возможных выбросов в данных. Значение медианы  будет объективнее описывать характерное значение данных.



In [19]:
# заполняем пропущенные значения медианными значениями по столбцам по категориям тип занятости
df['days_employed'] = df.groupby('income_type')['days_employed'].apply(lambda s: s.fillna(s.mean()))
df['total_income'] = df.groupby('income_type')['total_income'].apply(lambda s: s.fillna(s.mean()))

In [20]:
df.isna().sum() # проверяем отсутствие пропусков в данных

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных.

Заменим типы данных в столбцах `days_employed` и `total_income` на целочисленный, с помощью метода astype().

In [21]:
df['days_employed'] = df['days_employed'].astype('int') #изменяем тип данных в столбце `days_employed`

In [22]:
df['total_income'] = df['total_income'].astype('int') #изменяем тип данных в столбце 'total_income'


In [23]:
df.info() #проверяем внесенные изменения

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int32 
 1   days_employed     21525 non-null  int32 
 2   dob_years         21525 non-null  int32 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int32 
 11  purpose           21525 non-null  object
dtypes: int32(4), int64(3), object(5)
memory usage: 1.6+ MB


### Удаление дубликатов.

В таблице могут присутствовать дубликаты строк. Проверим их наличие и подсчитаем количество.

In [24]:
df.duplicated().sum() # подсчет явных дабликатов

54

In [25]:
df = df.drop_duplicates().reset_index(drop=True) # удаление явных дубликатов (с удалением старых индексов и формированием новых)
df.duplicated().sum() # проверка наличия явных дубликатов

0

Были удалены все явные дубликаты, но в данных могли остаться неявные дубликаты. Проверим значения по столбцам `education`, `family_status` и `income_type`.

In [26]:
df['education'].unique() # вывод уникальных значений столбца 'education'

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

В списке 'education' присутствуют неявные дубликаты: различное написание одного и того же уровня образования строчными и заглавными буквами. Такие дубликаты могли появиться из-за ошибок во время ввода данных.
Приведем все символы в строке к нижнему регистру.

In [27]:
df['education'] = df['education'].str.lower()
df['education'].unique() #проверка исправленных значений в столбце'education'

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [28]:
df['family_status'].unique() #вывод уникальных значений 'family_status'

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

В списке 'family_status' неявных дубликатов не обнаружено

In [29]:
df['income_type'].unique() #вывод уникальных значений income_type

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

В списке 'family_status' неявных дубликатов не обнаружено

После исправления дубликатов в столбце 'education', проверим наличие дубликатов строк в таблице. При их обнаружении удалим строки дубликаты и проверим результат выполнения функции.

In [30]:
df.duplicated().sum() #подсчет дубликатов

17

In [31]:
df = df.drop_duplicates().reset_index(drop=True) # удаление дубликатов (с удалением старых индексов и формированием новых)
df.duplicated().sum() # проверка наличия дубликатов

0

### Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Создадим два новых датафрейма(словаря) со столбцами:
-	education_id и education — в первом;
-	family_status_id и family_status — во втором.


In [32]:
df_education = df[['education_id', 'education']] # создаем словарь 'education'
df_education = df_education.drop_duplicates().reset_index(drop=True) #удаляем дубликаты строк
df_education #проверяем словарь 'education'

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


In [33]:
df_family_status = df[['family_status_id', 'family_status']] # создаем словарь 'family_status'
df_family_status = df_family_status.drop_duplicates().reset_index(drop=True) #удаляем дубликаты строк
df_family_status #проверяем словарь 'education'

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


In [34]:
df.drop(columns = ['education', 'family_status'],axis = 1, inplace=True) # удаляем столбцы 'education', 'family_status' из исходной таблицы
df.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5475,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,12775,53,1,1,F,пенсионер,0,158616,сыграть свадьбу


### Категоризация дохода.

Общий доход клиентов напрямую связан с полученным образованием, рассчитаем диапазон зарплат исходя из уровня образования.


In [35]:
df['education_id'].value_counts() 

1    15172
0     5250
2      744
3      282
4        6
Name: education_id, dtype: int64

Распределим общий доход каждого клиента по 5 категориям в соответствии с уровнями образования.
Cоздадим столбец `total_income_category` с категориями:
-	0–30000 — 'E';
-	30001–50000 — 'D';
-	50001–200000 — 'C';
-	200001–1000000 — 'B';
-	1000001 и выше — 'A'.

Для этого напишем функцию income_category(), которая принимает на вход один параметр 'income' и возвращает категорию заработной платы клиента. 
Применим функцию income_category() к столбцу `total_income` и таким образом присвоим каждому клиенту категорию заработной платы.


In [36]:
def income_category(income):
    if 0 <= income <= 30000:
        return 'E'
    if 30001 <= income <= 50000:
        return 'D'
    if 50001 <= income <= 200000:
        return 'C'
    if 200001 <= income <= 1000000:
        return 'B'
    return 'A'

df['total_income_category'] = df['total_income'].apply(income_category)
df['total_income_category'].value_counts() #выведем статистику распределени клиентов по уровню доходов


C    15514
B     5543
D      350
A       25
E       22
Name: total_income_category, dtype: int64

### Категоризация целей кредита.

Вызовим метод unique() для столбца `purpose` и проверим все уникальные значения целей кредита.

In [37]:
df['purpose'].unique() #изучим значения столбца 'purpose'

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

В списке представлены 38 различных значений цели кредита, но по смыслу все значения можно разделить на 4 категории:
- ‘операции с автомобилем’,
- ‘операции с недвижимостью’,
- ‘проведение свадьбы’,
- ‘получение образования’.

Сформируем новый столбец `purpose_category`, заполним его с помощью функции purpose_category() с одним входным параметром 'purpose'.

In [38]:
def purpose_category (purpose):
    if 'автомоб' in purpose:
        return 'операции с автомобилем'
    if 'недвиж'in purpose or 'жиль' in purpose:
        return 'операции с недвижимостью'
    if 'свадьб' in purpose:
        return 'проведение свадьбы'
    if 'образ' in purpose:
        return 'получение образования'
df['purpose_category'] = df['purpose'].apply(purpose_category)
df['purpose_category'].value_counts() #выведем статистику по целям кредита


операции с недвижимостью    10811
операции с автомобилем       4306
получение образования        4013
проведение свадьбы           2324
Name: purpose_category, dtype: int64

**Выводы**


Предобработка обнаружила проблемы в данных:

- пропущенные и отрицательные значения,
- аномалии в данных,
- дубликаты — явные и неявные.

Удаление дубликатов и корректрировка аномалий позволит получить более точные результаты.
Сформированы 2 датафрейма-словаря с данными о семейном положении и уровне образования клиентов.
Уровень доходов клиентов был разделен на 5 категорий в соответствии с уровнем образования клиента.
Все представленные в данных цели кредита разделены по смыслу на 4 категории.


## Проверка гипотез

Для проверки гипотез 

In [39]:
def pivot_share(column):
    df_pivot = df.pivot_table(index=[column], columns=['debt'], values='total_income', aggfunc='count')
    df_pivot['share_debt'] = df.groupby(column)['debt'].mean()
    return df_pivot.sort_values(by='share_debt')

### Проверка первой гипотезы:
Количество детей заемщика влияет на возврат кредита в срок.
Используем функцию debt() для столбца 'children'.

In [40]:
pivot_share('children')


debt,0,1,share_debt
children,,,
5,9.0,NaN,0.000000
0,13096.0,1071.0,0.075598
3,303.0,27.0,0.081818
1,4410.0,445.0,0.091658
2,1858.0,194.0,0.094542
4,37.0,4.0,0.097561


**Выводы**

Гипотеза подтвердилась частично. Доля клиентов с детьми, имеющих задолженность по кредиту выше чем, у клиентов без детей. Прослеживается тенденция роста доли клиентов, имеющих задолженность с увеличением количества детей, но в тоже время у клиентов с 3 детьми доля должников ниже чем в остальных категориях. Для клиентов с 5 детьми однозначные выводы сделать невозможно из-за небольшого их количества. 


### Проверка второй гипотезы: 
Семейное положение заемщика влияет на возврат кредита в срок.  

Создадим функцию, которая принимает на вход один параметр - столбец для анализа данных. Функция рассчитывает долю клиентов, которые имели задодолженность по возврату кредита по группам. 

На выходе получаем таблицу с тремя столбцами - `total_count`, `debt_sum`, `share_debt` отсортированную в порядке возрастания доли клиентов должников по анализируемому столбцу.
Используем функцию debt() для столбца `family_status_id`.

In [41]:
def debt (column): #функция для расчета доли клиентов, которые имели задолженность по возврату кредита
    debt_share = pd.DataFrame()
    debt_share['total_count'] = df.groupby(column)['debt'].count() #общее число клиентов, разделенное по группам
    debt_share['debt_sum'] = df.groupby(column)['debt'].sum()      #суммарное число клиентов по группам, имеющих задолженность
    debt_share['share_debt'] = df.groupby(column)['debt'].mean()   #доля клиентов имеющих задолженность от общего числа клиентов по группам
    debt_share = debt_share.sort_values(by=('share_debt'))         #сортировка в порядке возрастания доли клиентов должников
    return debt_share 



In [42]:
debt_family_status = debt('family_status_id')
debt_family_status

,total_count,debt_sum,share_debt
family_status_id,,,
2,959,63,0.065693
3,1195,85,0.071130
0,12339,931,0.075452
1,4151,388,0.093471
4,2810,274,0.097509


Для наглядной интерпретации полученных результатов объединим полученные данные со словарем family_status.

In [43]:
debt_family_status = df_family_status.merge(debt_family_status, on='family_status_id', how='left',)
debt_family_status = debt_family_status.drop(columns=['family_status_id'])
debt_family_status.sort_values(by=('share_debt'))

,family_status,total_count,debt_sum,share_debt
2,вдовец / вдова,959,63,0.065693
3,в разводе,1195,85,0.071130
0,женат / замужем,12339,931,0.075452
1,гражданский брак,4151,388,0.093471
4,Не женат / не замужем,2810,274,0.097509


**Выводы**

Гипотеза подтвердилась. Максимальное количество клиентов, имеющих задолженность по кредиту приходится на группу 'не женатых/не замужних' и 'гражданский брак'. Меньше всего клиентов, имеющих задолженность по кредиту в группе 'вдовец/вдова'. Такие результаты могут быть связаны с возрастом клиентов, возможно люди более старшего возраста могут более ответственно подходить к вопросу возврата кредита в срок. 

Рассмотрим гипотезу влияния возраста клиентов на возврат кредита в срок. Распределим всех клиентов в соответствии с возрастом на 4 категории: до 18 - дети, от 19 до 44 - молодой возраст, от 45 до 59 - средний возраст, от 59 и старше - пожилой возраст.

In [44]:
def age_category(age):
    if age <= 18:
        return 'дети'  #добавила данную категорию для дополнительной проверки отсутствия некорректного возраста
    if 19 <= age <= 44:
        return 'молодой возраст'
    if 45 <= age <= 59:
        return 'средний возраст'
    return 'пожилой возраст'

df['age_category'] = df['dob_years'].apply(age_category)
df['age_category'].value_counts() #выведем статистику распределения клиентов по возрасту

молодой возраст    11707
средний возраст     7146
пожилой возраст     2601
Name: age_category, dtype: int64

In [45]:
pivot_share('age_category')

debt,0,1,share_debt
age_category,,,
пожилой возраст,2470,131,0.050365
средний возраст,6661,485,0.067870
молодой возраст,10582,1125,0.096096


Гипотеза подтвердилась. Доля заемщиков, имеющих задолженность по кредиту уменьшается с увеличением возраста. Больше всего должников в группе 'молодой возраст', а меньше всего в группе пожилой возраст.

### Проверка третьей гипотезы:	
Уровень дохода заемщика влияет на возврат кредита в срок. 

In [46]:
pivot_share('total_income_category')


debt,0,1,share_debt
total_income_category,,,
D,329,21,0.060000
B,5157,386,0.069637
A,23,2,0.080000
C,14184,1330,0.085729
E,20,2,0.090909


**Выводы**

Гипотеза не подтвердилась. Уровень дохода заемщика не влияет на возврат кредита в срок. Меньше всего клиентов, имеющих задолженность по кредиту в группе клиентов с доходом ниже среднего, а больше всего в группе с самым низким доходом, но количество этих клиентов крайне мало, что не позволяет сделать объективных выводов.

### Проверка четвертой гипотезы:	
Как разные цели кредита влияют на его возврат в срок. 

In [47]:
pivot_share('purpose_category')

debt,0,1,share_debt
purpose_category,,,
операции с недвижимостью,10029,782,0.072334
проведение свадьбы,2138,186,0.080034
получение образования,3643,370,0.092200
операции с автомобилем,3903,403,0.093590


**Выводы**

Меньше всего задолженностей по кредиту в категории 'операции с недвижимостью', чуть больше доля должников по кредиту в категории 'проведение свадьбы'. Самая большая доля клиентов, имеющих задолженность в категории 'получение образования' и 'операции с автомобилем'. По этим двум категориям доли должников почти равны. 	

## Общий вывод:

При обзоре данных были выявлены аномалии, пропуски значений. Были устранены следущие проблемы в данных: 
- пропущенные и отрицательные значения,
- аномалии в данных,
- дубликаты — явные и неявные.

Некорректные значения в столбце `days_employed` заменили на положительные числа , а значения аномального возраста и трудового стажа, а также аномальные значения  в столбце `children` заполнили логически подходящими медианны данными.
Рассчитав среднее и медиану по данным приняли решение заполнить пропущенные значения в данных медианой.
Привели типы данных к наиболее подходящему целочисленному типу.
После проверки проверки были удалены 54 явных дубликата, проверили столбцы на наличие неявных дубликатов. Привели данные в столбце `education`  к однообразному виду и после этого обнаружили и удалили еще 17 дубликатов.
Сформировали 2 дополнительных датафрейма-словаря для упрощения работы и наглядного вывода результатов.
Категоризировали доход клиентов исходя из уровня их образования, результаты занесли в новый столбец `total_income_category`. Объединили цели кредита в 4 основные категории, результаты занесли в столбец `purpose_category`.

Были проверены 4 гипотезы, исходя из имеющихся данных установлено:
1. Гипотеза подтвердилась частично для клиентов с 1,2,4 детьми, для клиентов с 3 и 5 детьми одонозначный вывод сделать нельзя.
2. Гипотеза подтвердилась. Семейное положение влияет на возврат кредита в срок. Меньше всего доля клиентов, имеющих задолженность у банка в группе 'вдовец / вдова', больше всего доля в группе 'Не женат / не замужем'.
3. Гипотеза не подтвердилась. Уровень дохода заемщика не влияет на возврат кредита в срок. 
4. Меньше всего задолженностей по кредиту в категории 'операции с недвижимостью', чуть больше доля должников по кредиту в категории 'проведение свадьбы'. Самая большая доля клиентов, имеющих задолженность в категории 'получение образования' и 'операции с автомобилем'. По этим двум категориям доли должников почти равны.

Взаимосвязь семейного положения клиентов и возврат кредита в срок может быть обусловлена в большей степени возрастом клиента, а не его семейным положением. В качестве дополнения была рассмотрена зависимость возврата кредита в срок от возраста клиентов. Она полностью подтверждена. Возраст клиента влияет на возврат кредита в срок.


